The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

```bash
cd $HOME
mkdir students
wget https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv
find students -name "LCP_22-23_students.csv"
cp LCP_22-23_students.csv students/LCP_22-23_students.csv
```

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

```bash
grep "PoD" LCP_22-23_students.csv > from_PoD.csv
grep "Physics" LCP_22-23_students.csv > from_Physics.csv
```

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

```bash
for letter in {a..z}; do echo -n "$letter: "; grep -c -i "^$letter" LCP_22-23_students.csv; done;
```

1\.d Find out which is the letter with most counts.

```bash
count_max=0
for letter in {a..z}; do
    count=$(grep -c -i "^$letter" LCP_22-23_students.csv);
    if [ $count -gt $count_max ]; then
        count_max=$count;
        letter_max=$letter;
        fi;
    done;
echo "the letter with most count is '$letter_max' with '$count_max' occurences"
```

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

```bash
i=0
sed -n '4p' LCP_22-23_students.csv
```

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

```bash
# cat data.csv | sed 's/,/ /g' > data.txt
grep -v "^#" data.csv | sed 's/,//g' > data.txt
```

2\.b How many even numbers are there?

```bash
i=0
for n in $(grep -oE '[0-9]+' data.txt); do
    if ((n%2==0)); then
        ((i++))
    fi
done

echo $i
```

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

```bash
ts=$(echo '100*(e(l(3)*0.5))/2' | bc -l)
inside1=0
inside2=0
i=0
for line in $(cat data.txt); do
    for n in $line; do
        ((i++))
        if   [ $i -eq 1 ]; then
            X1=$n
        elif [ $i -eq 2 ]; then
            Y1=$n
        elif [ $i -eq 3 ]; then
            Z1=$n
        elif [ $i -eq 4 ]; then
            X2=$n
        elif [ $i -eq 5 ]; then
            Y2=$n
        elif [ $i -eq 6 ]; then
            Z2=$n
            #
            R1=$(echo "sqrt($X1^2 + $Y1^2 + $Z1^2)" | bc -l)
            R2=$(echo "sqrt($X2^2 + $Y2^2 + $Z2^2)" | bc -l)
            if (( $(echo "$R1 <= $ts" | bc -l) )); then
                ((inside1++))
            fi
            if (( $(echo "$R2 <= $ts" | bc -l) )); then
                ((inside2++))
            fi
            #
            i=0
        fi
        #
    done
    #
done
```

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

```bash
echo "write n:"
read n
#
for ((i=1; i<=$n; i++)); do
    out_file="data_$i.txt"
    while read -r line; do
        line_tmp=""
        for k in $line; do
            w=$(echo "scale=3; $k / $i" | bc -l)
            line_tmp="$line_tmp $w"
        done
        echo "$line_tmp" >> "$out_file"
    done < data.txt
done
```